In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#Booking.com URL
url = 'https://www.booking.com/searchresults.en-gb.html?ss=India&ssne=India&ssne_untouched=India&label=en-in-booking-desktop-SoQWfYhAMBURf0HSQntj1AS652796016141%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-334108349%3Alp9061996%3Ali%3Adec%3Adm&aid=2311236&lang=en-gb&sb=1&src_elem=sb&src=index&dest_id=98&dest_type=country&checkin=2023-09-01&checkout=2023-09-03&group_adults=2&no_rooms=1&sb_travel_purpose=leisure&selected_currency=INR'

#URL to get my device user agent: https://httpbin.org/get
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36", 
    "X-Amzn-Trace-Id": "Root=1-64dc9fa7-43afb96a4334f8a707a66ae1"}

In [3]:
#Scraping 1st time to find total number of pages
response  = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser') 

no_of_pages = int(soup.find_all('button', 'a83ed08757 a2028338ea')[-1].text)

In [4]:
#Extracting hotel listed for all pages
hotels_data = []
for i in range(0, 10): #For last value of range, need to use no_of_pages, here I am using a small value to fetch data quickly. Fetching till page 5.
    #url = 'https://www.booking.com/searchresults.en-gb.html?ss=India&ssne=India&ssne_untouched=India&label=en-in-booking-desktop-SoQWfYhAMBURf0HSQntj1AS652796016141%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-334108349%3Alp9061996%3Ali%3Adec%3Adm&aid=2311236&lang=en-gb&sb=1&src_elem=sb&src=index&dest_id=98&dest_type=country&checkin=2023-09-01&checkout=2023-09-03&group_adults=2&no_rooms=1&sb_travel_purpose=leisure&selected_currency=INR'
    offset=25*i #each pages showing 25 records
    url = url+'&offset='+str(offset)
    #print(url)
    response  = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')        
    #print(soup.prettify())
        
    #All hotel in every page    
    hotels = soup.find_all('div', {'data-testid': 'property-card'})
    #print(len(hotels))
    
    # Loop over the hotel elements and extract the desired data
    for hotel in hotels:
        #Extract hotel link
        link = hotel.find('a', {'data-testid': 'title-link'}).get('href')
        
        # Extract the hotel name
        name_element = hotel.find('div', {'data-testid': 'title'})
        name = name_element.text.strip()
        #print(name)
        
        # Extract the hotel location
        location_element = hotel.find('span', {'data-testid': 'address'})
        location = location_element.text.strip()
        
        # Extract the hotel price
        price_element = hotel.find('span', {'data-testid': 'price-and-discounted-price'})
        price = price_element.string.strip().replace(',', '')
        price = price[4:]
        
        # Extract the hotel rating
        rating_element = hotel.find('div', {'class': 'a3b8729ab1 d86cee9b25'})
        if rating_element:
            rating = float(rating_element.text.strip())
        else:
            rating = 0.0
    
        # Append hotels_data with info about hotel, remove duplicate if occurs
        if {'Name': name, 'Location': location, 'Price': price, 'Rating': rating} not in hotels_data:
            hotels_data.append({
                'Hotel Link': link,
                'Name': name,
                'Location': location,
                'Price': price,
                'Rating': rating
            })
        
hotels = pd.DataFrame(hotels_data)
hotels.to_csv('booking.com_hotels.csv', header=True, index=False)

In [12]:
file=pd.read_csv(r'C:\Users\sd25753\Documents\booking.com_hotels.csv')
file

,Location,Name,Price,Rating
0,Sheoganj,Sun Stone Hotel & Club - Jawai,6406,8.9
1,Kāhārpāra,Hotel Aashiyana,4659,7.1
2,Guman,"CZAR Palace Resort, Udaipur",16800,4.7
3,Karad,Sapphire Premium Hotel & Suite,4949,7.5
4,Kevadia,"Comfort Inn , Statue of unity",8960,7.0
5,Gangtok,Muscatel Grand Silkroute - 300 Mts from MG Marg,11365,8.2
6,Kānha,Sterling Kanha,13033,8.0
7,Bagar,Neemrana's - Piramal Haveli,8960,7.9
8,Kondapur,FabHotel Corner Courtyard,3090,8.1
9,Mussoorie,"Nature Valley Resort, Mussoorie - By Sitara Group",4198,8.7
